<a href="https://colab.research.google.com/github/angalponangel-spec/IESS_HTMC_TIC/blob/main/Copia_de_CRUD_v6_PRO_ULTRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import psycopg2
from psycopg2 import OperationalError
from datetime import datetime


DB_CONFIG = {
    "host": "localhost",
    "database": "iafi01_db",
    "user": "postgres",
    "password": "1234",
    "port": 5432
}

USUARIOS = {
    "angel": {"password": "1234", "rol": "ADMIN"},
    "medico1": {"password": "abcd", "rol": "EDITOR"},
    "consulta1": {"password": "view", "rol": "LECTOR"},
}
st.set_page_config(page_title="IESS | IAFI01 PostgreSQL", layout="wide")

if "usuario" not in st.session_state:
    st.session_state["usuario"] = None
if "rol" not in st.session_state:
    st.session_state["rol"] = None

# --------------------------------------------------
# 2. FUNCIÓN DE LOGIN
# --------------------------------------------------
def login():
    st.title("🔐 Acceso al Sistema Hospitalario")
    with st.form("login_form"):
        user = st.text_input("Usuario")
        pw = st.text_input("Contraseña", type="password")
        submit = st.form_submit_button("Ingresar")

        if submit:
            if user in USUARIOS and USUARIOS[user]["password"] == pw:
                st.session_state["usuario"] = user
                st.session_state["rol"] = USUARIOS[user]["rol"]
                st.success("Acceso concedido")
                st.rerun() # Esto recarga el código ya con la sesión activa
            else:
                st.error("Credenciales incorrectas")

# --------------------------------------------------
# 3. FILTRO DE SEGURIDAD (El "Gatekeeper")
# --------------------------------------------------
if st.session_state["usuario"] is None:
    login()
    st.stop()  # ✋ ¡ALTO! Si no hay login, no lee nada de lo que sigue



# --------------------------------------------------
# CONEXIÓN
# --------------------------------------------------

def conectar_db():
    try:
        return psycopg2.connect(**DB_CONFIG)
    except OperationalError as e:
        st.error(f"Error de conexión: {e}")
        st.stop()

# --------------------------------------------------
# FUNCIONES DB
# --------------------------------------------------

def cargar_datos():
    conn = conectar_db()
    df = pd.read_sql("SELECT * FROM iafi01 ORDER BY iafhis", conn)
    conn.close()
    return df

def insertar_registro(datos):
    conn = conectar_db()
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO iafi01
        (iafhis, iafnom, iafced, iaffna, iafsts, iafdat, iafsex, iaftaf, iafser)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, datos)
    conn.commit()
    cur.close()
    conn.close()

def actualizar_registro(historia, nombre, sexo):
    conn = conectar_db()
    cur = conn.cursor()

    cur.execute("SELECT iafnom, iafsex FROM iafi01 WHERE iafhis=%s", (historia,))
    anterior = cur.fetchone()

    if not anterior:
        cur.close()
        conn.close()
        return None

    cur.execute("""
        UPDATE iafi01
        SET iafnom=%s, iafsex=%s
        WHERE iafhis=%s
    """, (nombre.upper(), sexo, historia))

    conn.commit()
    cur.close()
    conn.close()
    return anterior

def baja_logica(historia):
    conn = conectar_db()
    cur = conn.cursor()
    cur.execute("UPDATE iafi01 SET iafsts='I' WHERE iafhis=%s", (historia,))
    conn.commit()
    cur.close()
    conn.close()

def registrar_auditoria(his, campo, antes, despues, usuario, accion):
    conn = conectar_db()
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO auditoria
        (iafhis, campo, valor_antes, valor_despues, usuario, fecha, accion)
        VALUES (%s,%s,%s,%s,%s,%s,%s)
    """, (his, campo, str(antes), str(despues), usuario, datetime.now(), accion))
    conn.commit()
    cur.close()
    conn.close()

def cargar_auditoria():
    conn = conectar_db()
    df = pd.read_sql("SELECT * FROM auditoria ORDER BY fecha DESC", conn)
    conn.close()
    return df

# --------------------------------------------------
# LOGIN
# --------------------------------------------------

def login():
    st.sidebar.subheader("🔐 Inicio de sesión")
    usuario = st.sidebar.text_input("Usuario")
    password = st.sidebar.text_input("Contraseña", type="password")

    if st.sidebar.button("Ingresar"):
        if usuario in USUARIOS and USUARIOS[usuario]["password"] == password:
            st.session_state["usuario"] = usuario
            st.session_state["rol"] = USUARIOS[usuario]["rol"]
            st.rerun()
        else:
            st.error("Credenciales inválidas")

if "usuario" not in st.session_state:
    login()
    st.stop()

usuario = st.session_state["usuario"]
rol = st.session_state["rol"]

st.sidebar.markdown(f"👤 Usuario: {usuario}")
st.sidebar.markdown(f"🎖 Rol: {rol}")

if st.sidebar.button("🔌 Cerrar sesión"):
    st.session_state.clear()
    st.rerun()

# --------------------------------------------------
# MENÚ
# --------------------------------------------------

if rol == "LECTOR":
    opcion = st.sidebar.radio("Menú", ["📄 Consultar"])
elif rol == "EDITOR":
    opcion = st.sidebar.radio("Menú", ["📄 Consultar", "➕ Alta", "✏️ Modificación"])
else:
    opcion = st.sidebar.radio("Menú", ["📄 Consultar", "➕ Alta", "✏️ Modificación", "❌ Baja", "🕵️ Auditoría"])

st.title("🏥 IESS - Sistema IAFI01 (PostgreSQL PRO)")

# --------------------------------------------------
# CONSULTAR
# --------------------------------------------------

if opcion == "📄 Consultar":
    st.dataframe(cargar_datos(), use_container_width=True)

# --------------------------------------------------
# ALTA
# --------------------------------------------------

elif opcion == "➕ Alta":

    his = st.number_input("Historia Clínica", step=1)
    nombre = st.text_input("Nombre")
    cedula = st.text_input("Cédula")
    sexo = st.selectbox("Sexo", ["M","F"])

    if st.button("Guardar"):
        insertar_registro((
            his,
            nombre.upper(),
            cedula,
            datetime.now(),
            "A",
            datetime.now(),
            sexo,
            "GENERAL",
            "SERVICIO"
        ))

        registrar_auditoria(his, "ALTA", "", nombre, usuario, "ALTA")
        st.success("Registro creado correctamente.")

# --------------------------------------------------
# MODIFICACIÓN
# --------------------------------------------------

elif opcion == "✏️ Modificación":

    df = cargar_datos()
    activos = df[df["iafsts"]=="A"]

    if activos.empty:
        st.warning("No existen registros activos.")
        st.stop()

    his = st.selectbox("Historia", activos["iafhis"].tolist())
    registro = activos[activos["iafhis"]==his].iloc[0]

    nuevo_nombre = st.text_input("Nombre", registro["iafnom"])
    nuevo_sexo = st.selectbox("Sexo", ["M","F"], index=0 if registro["iafsex"]=="M" else 1)

    if st.button("Actualizar"):
        anterior = actualizar_registro(his, nuevo_nombre, nuevo_sexo)

        if anterior:
            if anterior[0] != nuevo_nombre:
                registrar_auditoria(his,"iafnom",anterior[0],nuevo_nombre,usuario,"MODIFICACION")

            if anterior[1] != nuevo_sexo:
                registrar_auditoria(his,"iafsex",anterior[1],nuevo_sexo,usuario,"MODIFICACION")

            st.success("Registro actualizado correctamente.")

# --------------------------------------------------
# BAJA
# --------------------------------------------------

elif opcion == "❌ Baja":

    df = cargar_datos()
    activos = df[df["iafsts"]=="A"]

    if activos.empty:
        st.warning("No existen registros activos.")
        st.stop()

    his = st.selectbox("Historia a dar de baja", activos["iafhis"].tolist())

    if st.button("Confirmar Baja"):
        baja_logica(his)
        registrar_auditoria(his,"iafsts","A","I",usuario,"BAJA")
        st.success("Registro dado de baja correctamente.")

# --------------------------------------------------
# AUDITORÍA
# --------------------------------------------------

elif opcion == "🕵️ Auditoría":
    st.dataframe(cargar_auditoria(), use_container_width=True)


2026-02-13 10:37:21.337 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-13 10:37:21.337 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-13 10:37:21.338 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2026-02-13 10:37:21.338 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-13 10:37:21.339 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-13 10:37:21.339 WARNING streamlit.runtime.scriptrunner_utils.script_run_c